In [1]:
import torch
#from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
#from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data = pd.read_csv('/content/text_emotion.csv').head(10000)
data.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   10000 non-null  int64 
 1   sentiment  10000 non-null  object
 2   author     10000 non-null  object
 3   content    10000 non-null  object
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


In [4]:
data.drop(['tweet_id','author'], axis = 1, inplace = True)

In [5]:
x = data['content'].map(lambda content: re.sub('[^a-zA-Z\s]+', ' ', content).lower())
y = data['sentiment']
y
#text = data['content']


0            empty
1          sadness
2          sadness
3       enthusiasm
4          neutral
           ...    
9995       sadness
9996         worry
9997       neutral
9998         anger
9999          hate
Name: sentiment, Length: 10000, dtype: object

In [6]:
'''
# Reverse one-hot encoding
for index, row in data.iterrows():
    for column in data.columns[:-1]:  # Exclude the 'emotion' column
        if row[column] == 1:
            data.at[index, 'emotion'] = column
            '''

"\n# Reverse one-hot encoding\nfor index, row in data.iterrows():\n    for column in data.columns[:-1]:  # Exclude the 'emotion' column\n        if row[column] == 1:\n            data.at[index, 'emotion'] = column\n            "

In [7]:
from sklearn.preprocessing import LabelEncoder
# Encode labels as integers
label_encoder = LabelEncoder()
label = label_encoder.fit_transform(data['sentiment'])


In [8]:
# Split the data
#x = data['text']
train_text, val_text, train_label, val_label = train_test_split(x, label, test_size=0.2, random_state=42)

In [9]:
#!pip install wandb
#!pip install -U transformers accelerate
import wandb
# Define the RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(set(label)))

# Tokenize the data
train_encodings = tokenizer(list(train_text), truncation=True, padding='max_length', return_tensors='pt', max_length=128)
val_encodings = tokenizer(list(val_text), truncation=True, padding='max_length', return_tensors='pt', max_length=128)

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Convert label to long
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_label)
val_dataset = SentimentDataset(val_encodings, val_label)

training_args = TrainingArguments(
    output_dir='./roberta-output',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=500,
    num_train_epochs=8,
    logging_dir='./logs_roberta',
    logging_steps=100,
    evaluation_strategy="epoch",
    report_to="wandb"
)

# Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": accuracy_score(labels, preds)}

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()

# Access the accuracy from the evaluation results
accuracy = eval_results["eval_accuracy"]
print(f"Accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,1.945400,2.016146,0.310000
2,1.942500,1.989064,0.310000
3,1.938400,1.993718,0.310000
4,1.970000,1.978904,0.310000
5,1.951400,1.982078,0.310000
6,1.963400,1.977847,0.310000
7,1.955200,1.977737,0.310000
8,1.927300,1.964559,0.316000


Accuracy: 0.316


In [10]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')



ValueError: mount failed

In [11]:
output_dir = '/content/drive/My Drive'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/My Drive/tokenizer_config.json',
 '/content/drive/My Drive/special_tokens_map.json',
 '/content/drive/My Drive/vocab.json',
 '/content/drive/My Drive/merges.txt',
 '/content/drive/My Drive/added_tokens.json')

In [13]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# Initialize WandB for logging
import wandb
wandb.init(project="sentiment-analysis")
# Define the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(label)))

# Tokenize the data
train_encodings = tokenizer(list(train_text), truncation=True, padding='max_length', return_tensors='pt', max_length=128)
val_encodings = tokenizer(list(val_text), truncation=True, padding='max_length', return_tensors='pt', max_length=128)

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Convert label to long
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_label)
val_dataset = SentimentDataset(val_encodings, val_label)

training_args = TrainingArguments(
    output_dir='./bert-output',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=500,
    num_train_epochs=8,
    logging_dir='./logs_bert',
    logging_steps=100,
    evaluation_strategy="epoch",
    report_to="wandb"
)

# Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": accuracy_score(labels, preds)}

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()

# Access the accuracy from the evaluation results
accuracy = eval_results["eval_accuracy"]
print(f"Accuracy: {accuracy}")

# Save the model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

eval/accuracy,▁▁▁▁▁▁▁██
eval/loss,█▄▅▃▃▃▃▁▁
eval/runtime,▁▁▁▁▂▃▂▃█
eval/samples_per_second,███▇▇▆▇▆▁
eval/steps_per_second,████▇▆▇▆▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▄▂▅▅▃▃▃▂▃▃█▂▂▃▃▃▄▃▄▃▂▁▃▂▃▂▄▂▁▁▁▂▄▃▂▂▃▄▂▂
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▃▃▇▇▆▅▆▇▄▃▃▇▄▃▄▁▅▄▄▃▄▁▄▁▃▆▃▃▅▁▃▃▃█▂▃▃▂▂
eval/accuracy,0.316


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.862300,1.909542,0.337500
2,1.737300,1.829366,0.340500
3,1.444600,1.983775,0.358000
4,1.016400,2.348422,0.338500
5,0.590100,3.035544,0.310000
6,0.330200,3.714723,0.324000
7,0.186400,4.210448,0.328000
8,0.077500,4.426257,0.313500


Accuracy: 0.3135


('/content/drive/My Drive/tokenizer_config.json',
 '/content/drive/My Drive/special_tokens_map.json',
 '/content/drive/My Drive/vocab.txt',
 '/content/drive/My Drive/added_tokens.json')